# 💎 XGBoost Sniper v3: Obsidian Refinement

**Goal:** Create a hyper-refined model that limits volume to ~10 picks/day ('Sniper' approach) to maximize ROI.

**Methodology:**
1. **Fetch & Engineer:** Reuse the v2 Hybrid Momentum features.
2. **Train:** XGBoost Classifier.
3. **Optimize (NEW):** Instead of one pick per league, we implement a **Daily Portfolio Cap**.
   - Sort all daily candidates by **Expected Value (Edge * Prob)**.
   - Take the top `N` picks per day (e.g., Top 10).
   - Use stricter value floors (e.g., only bets with >5% edge).

In [1]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from supabase import create_client, Client
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score
import joblib
import warnings

# Load Environment Variables
load_dotenv()
warnings.filterwarnings('ignore')

# OPTIMIZATION FOR RYZEN 7 7700 (16 Threads)
N_JOBS = 16 

In [2]:
class SportsDataPipeline:
    def __init__(self):
        self.url: str = os.environ.get("SUPABASE_URL")
        self.key: str = os.environ.get("SUPABASE_KEY")
        if not self.url or not self.key:
            raise ValueError("❌ Missing Supabase credentials in .env file.")
        self.supabase: Client = create_client(self.url, self.key)
    
    def _fetch_all_batches(self, table_name, select_query="*", batch_size=1000):
        all_rows = []
        start = 0
        print(f"📥 Fetching '{table_name}'...", end=" ", flush=True)
        
        while True:
            end = start + batch_size - 1
            try:
                response = self.supabase.table(table_name).select(select_query).range(start, end).execute()
                data = response.data
                if not data: break
                
                all_rows.extend(data)
                if len(all_rows) % 5000 == 0: print(f"{len(all_rows)}...", end=" ", flush=True)
                if len(data) < batch_size: break
                start += batch_size
            except Exception as e:
                print(f"\n❌ Error fetching batch starting at {start}: {e}")
                break
        
        print(f"Done. ({len(all_rows)} rows)")
        return all_rows

    def fetch_master_data(self):
        # 1. Fetch Picks
        pick_cols = "id, pick_date, pick_value, unit, odds_american, result, capper_id, league_id, bet_type_id"
        picks_data = self._fetch_all_batches('picks', pick_cols)
        df_picks = pd.DataFrame(picks_data)
        if df_picks.empty: return pd.DataFrame()

        # 2. Fetch References
        cappers = pd.DataFrame(self._fetch_all_batches('capper_directory', "id, canonical_name"))
        leagues = pd.DataFrame(self._fetch_all_batches('leagues', "id, name, sport"))
        bet_types = pd.DataFrame(self._fetch_all_batches('bet_types', "id, name"))
        
        # 3. Merge
        print("🔄 Linking data relationships...")
        df = df_picks.merge(cappers, left_on='capper_id', right_on='id', how='left', suffixes=('', '_capper'))
        df = df.merge(leagues, left_on='league_id', right_on='id', how='left', suffixes=('', '_league'))
        
        if not bet_types.empty:
            df = df.merge(bet_types, left_on='bet_type_id', right_on='id', how='left', suffixes=('', '_bt'))
            df.rename(columns={'name_bt': 'bet_type_name'}, inplace=True)
        else:
            df['bet_type_name'] = 'unknown'

        # 4. Cleanup
        df['pick_date'] = pd.to_datetime(df['pick_date'])
        if 'name' in df.columns: df.rename(columns={'name': 'league_name'}, inplace=True)
        if 'sport' in df.columns: df.rename(columns={'sport': 'sport_name'}, inplace=True)
        
        # Filter Lotto Plays
        df['odds_american'] = pd.to_numeric(df['odds_american'], errors='coerce').fillna(-110)
        original_len = len(df)
        df = df[df['odds_american'] <= 250] 
        print(f"🧹 Filtered out {original_len - len(df)} 'Lotto' plays (+250 or higher).")
        
        return df.sort_values('pick_date')

# Load Data
pipeline = SportsDataPipeline()
raw_data = pipeline.fetch_master_data()

📥 Fetching 'picks'... 5000... 10000... 15000... 20000... 25000... 30000... 35000... 40000... 45000... 50000... 55000... 60000... 65000... 70000... 75000... 80000... 85000... Done. (87686 rows)
📥 Fetching 'capper_directory'... Done. (513 rows)
📥 Fetching 'leagues'... Done. (30 rows)
📥 Fetching 'bet_types'... Done. (13 rows)
🔄 Linking data relationships...
🧹 Filtered out 780 'Lotto' plays (+250 or higher).


In [3]:
class FeatureEngineer:
    def __init__(self, df):
        self.df = df.copy()
        
    def _american_to_decimal(self, odds):
        if pd.isna(odds) or odds == 0: return 1.91 
        if odds > 0: return (odds / 100) + 1
        return (100 / abs(odds)) + 1

    def _calculate_streaks(self, series):
        streaks = series.groupby((series != series.shift()).cumsum()).cumcount() + 1
        result_array = np.where(series == 0, -streaks, streaks)
        return pd.Series(result_array, index=series.index)

    def process_features(self):
        print("🛠️  Engineering Features (V3 Logic)...")
        df = self.df.copy()
        
        # 1. Standard Conversion
        df['unit'] = pd.to_numeric(df['unit'], errors='coerce').fillna(1.0)
        df['decimal_odds'] = df['odds_american'].apply(self._american_to_decimal)
        
        if 'result' in df.columns:
            res = df['result'].astype(str).str.lower().str.strip()
            conditions = [
                res.isin(['win', 'won', 'hit']),
                res.isin(['loss', 'lost', 'miss']),
                res.isin(['push', 'void', 'draw', 'tie'])
            ]
            df['outcome'] = np.select(conditions, [1.0, 0.0, 0.5], default=np.nan)
        
        conditions_roi = [df['outcome'] == 1.0, df['outcome'] == 0.0]
        choices_roi = [df['unit'] * (df['decimal_odds'] - 1), -df['unit']]
        df['profit_units'] = np.select(conditions_roi, choices_roi, default=0.0)
        
        df = df.sort_values(['capper_id', 'pick_date'])
        
        # 2. Rolling Stats (Capper)
        df['capper_experience'] = df.groupby('capper_id').cumcount()
        
        df = df.set_index('pick_date')
        grouped = df.groupby('capper_id')
        for window in ['7D', '30D']:
            s = window.lower()
            df[f'acc_{s}'] = grouped['outcome'].transform(lambda x: x.rolling(window, min_periods=1).mean().shift(1))
            df[f'roi_{s}'] = grouped['profit_units'].transform(lambda x: x.rolling(window, min_periods=1).sum().shift(1))
            df[f'vol_{s}'] = grouped['profit_units'].transform(lambda x: x.rolling(window, min_periods=1).std().shift(1))
        df = df.reset_index()

        # 3. Hotness & Streaks
        df['raw_hotness'] = df.groupby('capper_id')['profit_units']\
            .transform(lambda x: x.ewm(span=10, adjust=False).mean().shift(1))
            
        df['prev_outcome_binary'] = (df['outcome'] == 1.0).astype(int)
        df['streak_entering_game'] = df.groupby('capper_id')['prev_outcome_binary']\
            .transform(lambda x: self._calculate_streaks(x).shift(1))

        # 4. Consensus & Fade
        df['pick_norm'] = df['pick_value'].astype(str).str.lower().str.strip()
        df['consensus_count'] = df.groupby(['pick_date', 'league_name', 'pick_norm'])['capper_id'].transform('count')
        df['market_volume'] = df.groupby(['pick_date', 'league_name'])['capper_id'].transform('count')
        df['consensus_pct'] = df['consensus_count'] / (df['market_volume'] + 1)
        df['fade_score'] = (1 - df['consensus_pct']) * df['decimal_odds']

        # 5. Momentum Logic
        MOMENTUM_SPORTS = ['NBA', 'NCAAB', 'NHL', 'UFC']
        df['is_momentum_sport'] = df['league_name'].isin(MOMENTUM_SPORTS).astype(int)
        df['x_valid_hotness'] = df['raw_hotness'] * df['is_momentum_sport']

        # 6. League ROI
        df = df.sort_values('pick_date')
        df['league_rolling_roi'] = df.groupby('league_name')['profit_units']\
            .transform(lambda x: x.rolling(window=200, min_periods=20).mean().shift(1)).fillna(0)

        # 7. Probability Features
        df['implied_prob'] = 1 / df['decimal_odds']
        if 'bet_type_name' in df.columns:
            df['bet_type_code'] = df['bet_type_name'].astype('category').cat.codes
        else:
            df['bet_type_code'] = 0
            
        return df

engineer = FeatureEngineer(raw_data)
processed_data = engineer.process_features()

🛠️  Engineering Features (V3 Logic)...


In [4]:
class StrategyOptimizerV3:
    def __init__(self, df):
        self.df = df.copy()
        self.features = [
            'acc_7d', 'roi_7d', 'vol_7d', 'acc_30d', 'roi_30d', 'vol_30d',
            'capper_experience', 'consensus_count', 'implied_prob', 'bet_type_code',
            'raw_hotness', 'streak_entering_game', 
            'league_rolling_roi', 'fade_score',
            'is_momentum_sport', 'x_valid_hotness'
        ]
        self.CORE_LEAGUES = ['NBA', 'NCAAB', 'NFL', 'NCAAF', 'NHL', 'UFC']

    def run_optimization(self):
        print("🚀 STARTING V3 PORTFOLIO OPTIMIZATION...")
        
        # 1. Filter Data (Core Leagues)
        df_clean = self.df[self.df['outcome'].isin([0.0, 1.0])].copy()
        # Note: We keep all leagues for now to be safe, or filter to core
        df_clean = df_clean[df_clean['league_name'].isin(self.CORE_LEAGUES)]
        df_clean = df_clean.dropna(subset=self.features).sort_values('pick_date')
        
        # 2. Train Model
        split = int(len(df_clean) * 0.75) # 75% train to get more recent test data
        train_df = df_clean.iloc[:split]
        val_df = df_clean.iloc[split:].copy()
        
        print(f"   • Training Model on {len(train_df)} rows...")
        model = xgb.XGBClassifier(
            n_estimators=1200, learning_rate=0.015, max_depth=5, 
            n_jobs=16, tree_method='hist', random_state=42,
            early_stopping_rounds=50, subsample=0.8, colsample_bytree=0.8
        )
        model.fit(
            train_df[self.features], train_df['outcome'].astype(int),
            eval_set=[(val_df[self.features], val_df['outcome'].astype(int))],
            verbose=False
        )
        
        # Save Model
        joblib.dump(model, '../models/v3_obsidian.pkl')
        
        # Predictions
        val_df['prob'] = model.predict_proba(val_df[self.features])[:, 1]
        val_df['edge'] = val_df['prob'] - val_df['implied_prob']
        
        # 3. GLOBAL GRID SEARCH (Portfolio Builder)
        # We test stricter floors to ensure quality > quantity
        edge_floors = [0.03, 0.05, 0.08] 
        exp_floors = [10]
        daily_caps = [5, 10, 15] # Top N picks per day
        
        results = []
        best_roi = -999
        best_df = None
        
        for min_exp in exp_floors:
            for min_edge in edge_floors:
                for cap in daily_caps:
                    
                    # 1. Base Filter
                    candidates = val_df[
                        (val_df['capper_experience'] >= min_exp) &
                        (val_df['edge'] >= min_edge) &
                        (val_df['decimal_odds'] >= 1.70) # Min Odds Floor (-143)
                    ].copy()
                    
                    if candidates.empty: continue

                    # 2. Portfolio Construction (Top N Daily)
                    # Rank by Raw Edge (Value)
                    candidates = candidates.sort_values(['pick_date', 'edge'], ascending=[True, False])
                    
                    # Take top 'cap' picks per day across ALL leagues
                    portfolio = candidates.groupby('pick_date').head(cap)
                    
                    # 3. Calculate Stats
                    profit = portfolio['profit_units'].sum()
                    volume = len(portfolio)
                    roi = (profit / volume) * 100 if volume > 0 else 0
                    unique_days = portfolio['pick_date'].nunique()
                    bets_per_day = volume / unique_days if unique_days > 0 else 0
                    
                    results.append({
                        'Min_Exp': min_exp, 'Min_Edge': min_edge, 'Daily_Cap': cap,
                        'ROI': roi, 'Profit': profit, 'Bets': volume, 'Bets/Day': bets_per_day
                    })
                    
                    if roi > best_roi and volume > 50 and bets_per_day >= 3:
                        best_roi = roi
                        best_config = {'Min_Exp': min_exp, 'Min_Edge': min_edge, 'Daily_Cap': cap}
                        best_df = portfolio

        # Result Display
        res_df = pd.DataFrame(results).sort_values('ROI', ascending=False)
        print("\n🏆 TOP 5 CONFIGURATIONS:")
        print(res_df.head(5))
        
        if best_df is not None:
            print(f"\n✅ CHOSEN CONFIG: {best_config}")
            print(f"   • ROI: {best_roi:.2f}%")
            print(f"   • Total Profit: {best_df['profit_units'].sum():.2f}u")
            print(f"   • Bets/Day: {len(best_df)/best_df['pick_date'].nunique():.1f}")
            
            # Save Config for Inference
            import json
            config_save = {
                "Min_Exp": int(best_config['Min_Exp']),
                "Min_Edge": float(best_config['Min_Edge']),
                "Min_Odds": 1.70,
                "Daily_Cap": int(best_config['Daily_Cap']),
                "ROI": float(best_roi),
                "Profit": float(best_df['profit_units'].sum()),
                "Bets": int(len(best_df)),
                "Bets/Day": float(len(best_df)/best_df['pick_date'].nunique())
            }
            with open('../models/v3_config.json', 'w') as f:
                json.dump(config_save, f, indent=2)

            print("📁 Model and Config saved to models/v3_obsidian.pkl and models/v3_config.json")

optimizer = StrategyOptimizerV3(processed_data)
optimizer.run_optimization()

🚀 STARTING V3 PORTFOLIO OPTIMIZATION...
   • Training Model on 38708 rows...

🏆 TOP 5 CONFIGURATIONS:
   Min_Exp  Min_Edge  Daily_Cap        ROI      Profit  Bets   Bets/Day
6       10      0.08          5  49.056380   82.905281   169   4.970588
0       10      0.03          5  48.179577   81.905281   170   5.000000
3       10      0.05          5  48.179577   81.905281   170   5.000000
8       10      0.08         15  45.290740  217.848457   481  14.147059
2       10      0.03         15  42.808920  216.613135   506  14.882353

✅ CHOSEN CONFIG: {'Min_Exp': 10, 'Min_Edge': 0.08, 'Daily_Cap': 5}
   • ROI: 49.06%
   • Total Profit: 82.91u
   • Bets/Day: 5.0
📁 Model and Config saved to models/v3_obsidian.pkl and models/v3_config.json
